# 5 - Простая модель Encoder-Decoder для машинного перевода

*Источник: [туториал от bentrevett](https://github.com/bentrevett/pytorch-seq2seq)*

![](assets/seq2seq1.png)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import nltk
from nltk import word_tokenize
from datasets import Dataset, DatasetDict, load_metric
import torchtext
import tqdm
import evaluate
nltk.download('punkt')

In [ ]:
seed = 1234

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [ ]:
src_sentences, trg_sentences = [], []

with open('data/parallel_hotel_descriptions/data.txt') as f:
    for line in f:
        en, ru = line.strip().split('\t')
        trg_sentences.append(en)
        src_sentences.append(ru)

In [ ]:
len(src_sentences), len(trg_sentences)

In [ ]:
for src_sent, trg_sent in zip(src_sentences[:10], trg_sentences[:10]):
    print(src_sent)
    print(trg_sent)
    print('--------------------')

In [ ]:
src, trg = [], []

for src_sent, trg_sent in zip(src_sentences, trg_sentences):
    src.append(word_tokenize(src_sent))
    trg.append(word_tokenize(trg_sent))

In [ ]:
dataset = Dataset.from_dict({'src': src, 'trg': trg})

In [ ]:
dataset

In [ ]:
train_testvalid = dataset.train_test_split(test_size=0.2)
# Split the 10% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.7)
# gather everyone if you want to have a single DatasetDict
data = DatasetDict({
    'train': train_testvalid['train'],
    'validation': test_valid['train'],
    'test': test_valid['test']})

In [ ]:
data

In [ ]:
def preprocess_example(
    example,
    max_length,
    lower,
    sos_token,
    eos_token
):
    src_tokens = example["src"][:max_length]
    trg_tokens = example["trg"][:max_length]
    if lower:
        src_tokens = [token.lower() for token in src_tokens]
        trg_tokens = [token.lower() for token in trg_tokens]
    src_tokens = [sos_token] + src_tokens + [eos_token]
    trg_tokens = [sos_token] + trg_tokens + [eos_token]
    return {"src": src_tokens, "trg": trg_tokens}

In [ ]:
max_length = 1_000
lower = True
sos_token = "<sos>"
eos_token = "<eos>"

fn_kwargs = {
    "max_length": max_length,
    "lower": lower,
    "sos_token": sos_token,
    "eos_token": eos_token,
}


data = data.map(preprocess_example, fn_kwargs=fn_kwargs)

In [ ]:
data['train'][0]

In [ ]:
min_freq = 2
unk_token = "<unk>"
pad_token = "<pad>"

special_tokens = [
    unk_token,
    pad_token,
    sos_token,
    eos_token,
]

src_vocab = torchtext.vocab.build_vocab_from_iterator(
    data['train']["src"],
    min_freq=min_freq,
    specials=special_tokens,
)

trg_vocab = torchtext.vocab.build_vocab_from_iterator(
    data['train']["trg"],
    min_freq=min_freq,
    specials=special_tokens,  
)

In [ ]:
src_vocab.get_itos()[:15]

In [ ]:
trg_vocab.get_itos()[:15]

In [ ]:
src_vocab.get_itos()[42]

In [ ]:
trg_vocab.get_itos()[999]

In [ ]:
len(src_vocab), len(trg_vocab)

In [ ]:
"гостиница" in src_vocab, "переводчик" in src_vocab

In [ ]:
"The" in en_vocab

In [ ]:
assert src_vocab[unk_token] == trg_vocab[unk_token]
assert src_vocab[pad_token] == trg_vocab[pad_token]

unk_index = src_vocab[unk_token]
pad_index = src_vocab[pad_token]

In [ ]:
src_vocab.set_default_index(unk_index)
trg_vocab.set_default_index(unk_index)

In [ ]:
tokens = ["на", "летней", "школе", "свежий", "воздух", "и", "много", "программирования"]

In [ ]:
src_vocab.forward(tokens) # можно также src_vocab.lookup_indices(tokens)

In [ ]:
tokens = ["на", "летней", "школе", "свежий", "воздух"]

In [ ]:
src_vocab.lookup_tokens(src_vocab.forward(tokens))

In [ ]:
def numericalize_example(example, src_vocab, trg_vocab):
    src_ids = src_vocab.lookup_indices(example["src"])
    trg_ids = trg_vocab.lookup_indices(example["trg"])
    return {"src": trg_ids, "trg": trg_ids}

In [ ]:
fn_kwargs = {
    "src_vocab": src_vocab, 
    "trg_vocab": trg_vocab
}

transformed_data = data.map(numericalize_example, fn_kwargs=fn_kwargs)

In [ ]:
transformed_data['train'][0]

In [ ]:
data['train'][0]

In [ ]:
src_vocab.lookup_tokens(transformed_data['train'][0]["src"])

In [ ]:
data_type = "torch"
format_columns = ["src", "trg"]

transformed_data = transformed_data.with_format(
    type=data_type, 
    columns=format_columns, 
    output_all_columns=True
)

In [ ]:
transformed_data['train'][0]

In [ ]:
type(transformed_data['train'][0]['src'])

In [ ]:
def get_collate_fn(pad_index):
    
    def collate_fn(batch):
        batch_src_ids = [example["src"] for example in batch]
        batch_trg_ids = [example["trg"] for example in batch]
        batch_src_ids = nn.utils.rnn.pad_sequence(batch_src_ids, padding_value=pad_index)
        batch_trg_ids = nn.utils.rnn.pad_sequence(batch_trg_ids, padding_value=pad_index)
        batch = {
            "src": batch_src_ids,
            "trg": batch_trg_ids,
        }
        return batch
    
    return collate_fn

In [ ]:
def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    
    collate_fn = get_collate_fn(pad_index)
    
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle,
    )
    
    return data_loader

In [ ]:
batch_size = 32

train_data_loader = get_data_loader(transformed_data['train'], batch_size, pad_index, shuffle=True)
valid_data_loader = get_data_loader(transformed_data['validation'], batch_size, pad_index)
test_data_loader = get_data_loader(transformed_data['test'], batch_size, pad_index)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        # src = [src length, batch size]
        embedded = self.dropout(self.embedding(src))
        # embedded = [src length, batch size, embedding dim]
        outputs, (hidden, cell) = self.rnn(embedded)
        # outputs = [src length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # outputs are always from the top hidden layer
        return hidden, cell

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        # input = [batch size]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # n directions in the decoder will both always be 1, therefore:
        # hidden = [n layers, batch size, hidden dim]
        # context = [n layers, batch size, hidden dim]
        input = input.unsqueeze(0)
        # input = [1, batch size]
        embedded = self.dropout(self.embedding(input))
        # embedded = [1, batch size, embedding dim]
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        # output = [seq length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # seq length and n directions will always be 1 in this decoder, therefore:
        # output = [1, batch size, hidden dim]
        # hidden = [n layers, batch size, hidden dim]
        # cell = [n layers, batch size, hidden dim]
        prediction = self.fc_out(output.squeeze(0))
        # prediction = [batch size, output dim]
        return prediction, hidden, cell

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        assert encoder.hidden_dim == decoder.hidden_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio):
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        batch_size = trg.shape[1]
        trg_length = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        # tensor to store decoder outputs
        outputs = torch.zeros(trg_length, batch_size, trg_vocab_size).to(self.device)
        # last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # first input to the decoder is the <sos> tokens
        input = trg[0,:]
        # input = [batch size]
        for t in range(1, trg_length):
            # insert input token embedding, previous hidden and previous cell states
            # receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            # output = [batch size, output dim]
            # hidden = [n layers, batch size, hidden dim]
            # cell = [n layers, batch size, hidden dim]
            # place predictions in a tensor holding predictions for each token
            outputs[t] = output
            # decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            # get the highest predicted token from our predictions
            top1 = output.argmax(1)
            # if teacher forcing, use actual next token as next input
            # if not, use predicted token
            input = trg[t] if teacher_force else top1
            # input = [batch size]
        return outputs

In [ ]:
input_dim = len(src_vocab)
output_dim = len(trg_vocab)
encoder_embedding_dim = 256
decoder_embedding_dim = 256
hidden_dim = 512
n_layers = 2
encoder_dropout = 0.5
decoder_dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder = Encoder(
    input_dim,
    encoder_embedding_dim,
    hidden_dim,
    n_layers,
    encoder_dropout,
)

decoder = Decoder(
    output_dim,
    decoder_embedding_dim,
    hidden_dim,
    n_layers,
    decoder_dropout,
)

model = Seq2Seq(encoder, decoder, device).to(device)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"The model has {count_parameters(model):,} trainable parameters")

In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

In [ ]:
def train_fn(model, data_loader, optimizer, criterion, clip, teacher_forcing_ratio, device):
    model.train()
    epoch_loss = 0
    for i, batch in tqdm.tqdm(enumerate(data_loader)):
        src = batch["src"].to(device)
        trg = batch["trg"].to(device)
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        optimizer.zero_grad()
        output = model(src, trg, teacher_forcing_ratio)
        # output = [trg length, batch size, trg vocab size]
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        # output = [(trg length - 1) * batch size, trg vocab size]
        trg = trg[1:].view(-1)
        # trg = [(trg length - 1) * batch size]
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [ ]:
def evaluate_fn(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in tqdm.tqdm(enumerate(data_loader)):
            src = batch["src"].to(device)
            trg = batch["trg"].to(device)
            # src = [src length, batch size]
            # trg = [trg length, batch size]
            output = model(src, trg, 0) # turn off teacher forcing
            # output = [trg length, batch size, trg vocab size]
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            # output = [(trg length - 1) * batch size, trg vocab size]
            trg = trg[1:].view(-1)
            # trg = [(trg length - 1) * batch size]
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [ ]:
n_epochs = 10
clip = 1.0
teacher_forcing_ratio = 0.5

best_valid_loss = float("inf")

for epoch in range(n_epochs):
    print('Epoch 1')
    print('Training...')
    train_loss = train_fn(
        model, 
        train_data_loader, 
        optimizer, 
        criterion, 
        clip, 
        teacher_forcing_ratio, 
        device,
    )
    print('Evaluation...')
    valid_loss = evaluate_fn(
        model, 
        valid_data_loader, 
        criterion, 
        device,
    )

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), "tut1-model.pt")
    
    print(f"\tTrain Loss: {train_loss:7.3f} | Train PPL: {np.exp(train_loss):7.3f}")
    print(f"\tValid Loss: {valid_loss:7.3f} | Valid PPL: {np.exp(valid_loss):7.3f}")

In [ ]:
# model.load_state_dict(torch.load("tut1-model.pt"))

test_loss = evaluate_fn(model, test_data_loader, criterion, device)

print(f"| Test Loss: {test_loss:.3f} | Test PPL: {np.exp(test_loss):7.3f} |")

In [ ]:
def translate_sentence(
    sentence, 
    model,
    src_vocab,
    trg_vocab,
    lower,
    sos_token,
    eos_token,
    device,
    max_output_length=25,
):
    model.eval()
    with torch.no_grad():
        if isinstance(sentence, str):
            tokens = word_tokenize(sentence)
        else:
            tokens = [token for token in sentence]
        if lower:
            tokens = [token.lower() for token in tokens]
        tokens = [sos_token] + tokens + [eos_token]
        ids = src_vocab.forward(tokens)
        tensor = torch.LongTensor(ids).unsqueeze(-1).to(device)
        hidden, cell = model.encoder(tensor)
        inputs = trg_vocab.lookup_indices([sos_token])
        for _ in range(max_output_length):
            inputs_tensor = torch.LongTensor([inputs[-1]]).to(device)
            output, hidden, cell = model.decoder(inputs_tensor, hidden, cell)
            predicted_token = output.argmax(-1).item()
            inputs.append(predicted_token)
            if predicted_token == trg_vocab[eos_token]:
                break
        tokens = trg_vocab.lookup_tokens(inputs)
    return tokens

In [ ]:
sentence = " ".join(data['test'][42]["src"])
expected_translation = " ".join(data['test'][42]["trg"])

sentence, expected_translation

In [ ]:
translation = translate_sentence(
    sentence,
    model,
    src_vocab,
    trg_vocab,
    lower,
    sos_token,
    eos_token,
    device,
)

In [ ]:
' '.join(translation)

In [ ]:
sentence = "На летней школе свежий воздух и много программирования"

In [ ]:
translation = translate_sentence(
    sentence,
    model,
    src_vocab,
    trg_vocab,
    lower,
    sos_token,
    eos_token,
    device,
)

And we receive our translation...

In [ ]:
translation

We can now loop over our `test_data`, getting our model's translation of each test sentence.

In [ ]:
translations = [
    translate_sentence(
        example["src"],
        model,
        src_vocab,
        trg_vocab,
        lower,
        sos_token,
        eos_token,
        device,  
    ) for example in tqdm.tqdm(data['test'])
]

In [ ]:
bleu = load_metric("bleu")

In [ ]:
predictions = [
    translation[1:-1] for translation in translations
]

references = [
    [sentence[1:-1]] for sentence in data['test']['trg']
]

In [ ]:
predictions[0], references[0]

In [ ]:
results = bleu.compute(predictions=predictions, references=references)

In [ ]:
results